In [1]:
import pandas as pd
import math
from sklearn import preprocessing

In [2]:
school_data = pd.read_csv('buildbps.csv', encoding='latin1')
school_data.head()



,SMMA_Identifier,SMMA_Only_For_Map,BPS_School_Name,BPS_Historical_Name,SMMA_Abbreviated_Name,BPS_Address,BRA_Neighborhood,SMMA_latitude,SMMA_longitude,SMMA_Typology,...,SMMA_EA_K8_Adequacy_Cafeteria,SMMA_EA_K8_Adequacy_Stage,SMMA_EA_K8_Adequacy_Medical,SMMA_EA_K8_Adequacy_Administration,SMMA_EA_K8_Adequacy_Custodial,SMMA_EA_K8_Adequacy_Network,SMMA_EA_K8_Adequacy_Other_1,SMMA_EA_K8_Adequacy_Other_2,SMMA_EA_K8_Adequacy_Other_3,SMMA_EA_K8_Overall_EFE_spaces
0,031,NaN,"Adams, Samuel Elementary",Adams,Adams,"165 Webster St East Boston, MA 02128",East Boston,42.365553,-71.034917,Elementary School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,078,NaN,"Alighieri, Dante Montessori School",Alighieri,Alighieri,"37 Gove Street East Boston, MA 02128",East Boston,42.371565,-71.037608,Elementary School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,045A,NaN,Another Course to College*,Taft,ACC*,"20 Warren Street Brighton, MA 02135",Allston,42.350354,-71.145582,High School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,012,NaN,Baldwin Early Learning Pilot Academy,Baldwin ELC,Baldwin,"121 Corey Rd Brighton, MA 02135",Brighton,42.342037,-71.140529,Early Learning,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,087,NaN,"Bates, Phineas Elementary",Bates,Bates,"426 Beech St Roslindale, MA 02131",Roslindale,42.277663,-71.135353,Elementary School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def school_Classifier(row):
    #One-hot encoding for schools
    gradesk_6 = 0
    grades7_8 = 0
    grades9_12 = 0
    #k-6 schools actually have a large variety of grades
    #sometimes k0-5, sometimes k1-5, sometimes k0-8, sometimes the demonic k2-7
    #Ideally I would look at the grade column and split it into charactesrs, use that as the range to append to arrays
    education = row.at["SMMA_Educational_Program"]
    if education == "Elementary: Pre-K to 6":
        gradesk_6 = 1
        grades7_8 = 0
        grades9_12 = 0
    elif education == "Elementary: Pre-K to 8":
        gradesk_6 = 1
        grades7_8 = 1
        grades9_12 = 0
    elif education == "High Schools: 7 to 12":
        gradesk_6 = 0
        grades7_8 = 1
        grades9_12 = 1
    elif education == "High Schools: 9 to 12":
        gradesk_6 = 0
        grades7_8 = 0
        grades9_12 = 1
    else:
        #Ideally if the school got to this section of the code
        #I would use some sort of counter and tracker so I could know to look here if my data/results were weird
        schoolType = row.at["SMMA_Typology"]
        if schoolType == "Elementary School" or "Early Learning":
            gradesk_6 = 1
            grades7_8 = 0
            grades9_12 = 0
        elif schoolType == "K-8":
            gradesk_6 = 1
            grades7_8 = 1
            grades9_12 = 0
        elif schoolType == "Middle School":
            gradesk_6 = 0
            grades7_8 = 1
            grades9_12 = 0
        elif schoolType == "High School":
            gradesk_6 = 0
            grades7_8 = 0
            grades9_12 = 1
        else:
            gradesk_6 = 0
            grades7_8 = 0
            grades9_12 = 0
    row.at["gradesk_6"] = gradesk_6
    row.at["grades7_8"] = grades7_8
    row.at["grades9_12"] = grades9_12
    return row
def make_Ordinal(row):
    map_dictionary = {"Not Present":0, 
                      "Replace":1,
                      "Moderate":2,
                      "Minor":3,
                      "Adequate":4,
                      "Deficient":0,
                      "Poor":1,
                      "Fair":2,
                      "Good":3,
                      "Excellent":4,
                      "Not Observed":0,
                      "No":0,
                      "Yes":1,
                      "Not Listed":0,
                      "Listed":1,
                      "Phase 1":1,
                      "Phase 2":2,
                      "Phase 3":3
                      "Not Assessed": 0
                     }
    #Generally with "Not Assessed", the row won't have much data, so I could also drop the rows
    for index, value in row.iteritems():
        #checks for blank cells
        if type(value)== float and math.isnan(value):
            row.loc[index] = 0
        #switches ordinal values with integers
        if value in map_dictionary:
            row.loc[index] = map_dictionary[value]
    return row
def time(row):
    #use whatever python's time library is, and then convert the times to time since epoch
    pass
new_data = school_data.apply(make_Ordinal, axis = 1)
new_data = new_data.apply(school_Classifier, axis = 1)
print(new_data)

    SMMA_Identifier SMMA_Only_For_Map                       BPS_School_Name  \
0               031                 0              Adams, Samuel Elementary   
1               078                 0    Alighieri, Dante Montessori School   
2              045A                 0            Another Course to College*   
3               012                 0  Baldwin Early Learning Pilot Academy   
4               087                 0             Bates, Phineas Elementary   
..              ...               ...                                   ...   
136            011B                 0                 West Roxbury Academy*   
137            047B                 0      West Zone Early Learning Center*   
138             100                 0          Winship, F. Lyman Elementary   
139             073                 0             Winthrop, John Elementary   
140             044                 0    Young Achievers Science & Math K-8   

            BPS_Historical_Name SMMA_Abbreviated_Na